In [1]:
# Cell 3: Download and load QUASR device database
import gzip
import json
from io import BytesIO
import requests
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import time
import numpy as np
from simsopt.field import Current
from simsopt.geo import SurfaceRZFourier
from simsopt._core import load
from pathlib import Path
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import shutil

In [2]:
SIMSOPT_DIR = 'quasr_simsopt_files'
LOG_CSV = "canon_curr_quasr_log.csv"
data_dir = Path(SIMSOPT_DIR)
output_dir = Path('canon_curr_surface_coil_tfrecords')
os.makedirs(SIMSOPT_DIR, exist_ok=True)

MAX_COILS = 6
FEATURES_PER_COIL = 100
NUM_THREAD_WORKERS = 48
NUM_PROCESS_WORKERS = 1
CHUNK_SIZE = 10000
MAX_RETRIES = 5
RETRY_DELAY = 2  # seconds
MAX_NFP = 5
MAX_COEFS = 441

In [3]:
# url = "https://quasr.flatironinstitute.org/database.json.gz"
# print('Downloading device database...')
# r = requests.get(url)
# r.raise_for_status()

# with gzip.open(BytesIO(r.content), 'rt', encoding='utf-8') as f:
#     data = json.load(f)

# df = pd.DataFrame(**data)
# print(f"Loaded {len(df)} devices.")

# df.to_hdf('QUASR_Stellarators.h5', key = 'full_dataset')
df = pd.read_hdf('QUASR_Stellarators.h5', key = 'full_dataset')

In [4]:
# Cell 4: Apply filters to select matching devices
filtered = df[
    (df["Nfourier_coil"] == 16) &
    (df['qs_error'] >= -4) &
    # (df["max_elongation"] <= 10) &
    # (df["aspect_ratio"] >= 4) & (df["aspect_ratio"] <= 10) &
    (df["nc_per_hp"] >= 1) & (df["nc_per_hp"] <= 6) &
    (df["nfp"] >= 1) & (df["nfp"] <= 5)
].copy()

print(f"{len(filtered)} devices match your criteria.")

314309 devices match your criteria.


In [5]:
def simsopt_url(device_id):
    pid = device_id.zfill(7)
    return f"https://quasr.flatironinstitute.org/simsopt_serials/{pid[:4]}/serial{pid}.json"

In [6]:
# Cell 6: Robust download with retries
def download_with_retries(url: str, path: str) -> bool:
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = requests.get(url, timeout=30)
            if r.status_code == 200:
                with open(path, 'wb') as f:
                    f.write(r.content)
                return True
            else:
                print(f"{url} returned status {r.status_code} (attempt {attempt})")
        except Exception as e:
            print(f"Error on {url} (attempt {attempt}): {e}")
        time.sleep(RETRY_DELAY)
    return False

In [7]:
# Cell 7: Prepare log and list of device IDs to download
if os.path.exists(LOG_CSV):
    log_df = pd.read_csv(LOG_CSV, dtype=str)
else:
    log_df = pd.DataFrame(columns=["ID", 'simsopt_url', "status"])

processed = set(log_df["ID"])
device_ids = [str(d) for d in filtered["ID"] if str(d) not in processed] #this is where you change which df you want the device ids from
chunks = [device_ids[i:i+CHUNK_SIZE] for i in range(0, len(device_ids), CHUNK_SIZE)]
print(f"{len(device_ids)} devices to download in {len(chunks)} chunks.")

314309 devices to download in 32 chunks.


In [8]:
def process_device(dev_id):
        pid = dev_id.zfill(7)
        # vmec_path = os.path.join(VMEC_DIR, f"input.{pid}")
        simsopt_path = os.path.join(SIMSOPT_DIR, f"input_{pid}.json")
        # vmec_ok = os.path.exists(vmec_path) or download_with_retries(vmec_url(dev_id), vmec_path)
        simsopt_ok = os.path.exists(simsopt_path) or download_with_retries(simsopt_url(dev_id), simsopt_path)
        status = "success" if simsopt_ok else 'failed'
        return {
            "ID": dev_id,
            # "vmec_url": vmec_url(dev_id),
            'simsopt_url': simsopt_url(dev_id),
            "status": status
        }

In [9]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value.flatten()))

def _int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def serialize_data(id: str, coils: np.ndarray, coil_mask: np.ndarray, surface: np.ndarray, surface_mask: np.ndarray):
    feature = {
        'ID': _bytes_feature(id.encode('utf-8')),
        'coil_data': _float_feature(coils),
        'coil_mask': _int_feature(coil_mask),
        'surface_data': _float_feature(surface),
        'surface_mask': _int_feature(surface_mask)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [10]:
def sample_curve_points(curve, n_samples=200):
    """
    Return (N, 3) array of points on the curve, robust to simsoptpp vs Python API.
    - simsoptpp.CurveXYZFourier.gamma(): no-arg, uses internal grid
    - Python CurveXYZFourier.gamma(s): accepts an s-array
    """
    # Try simsoptpp form (no-arg)
    try:
        # (Optional) try to set resolution if the curve exposes a grid attribute.
        # Some builds have `curve.quadpoints` you can assign; if not, we just use whatever it has.
        pts = curve.gamma()               # should be (N,3) or (3,N)
    except TypeError:
        # Fall back to Python form that accepts s
        s_vals = np.linspace(0.0, 1.0, n_samples, endpoint=False)
        pts = curve.gamma(s_vals)

    pts = np.asarray(pts)
    if pts.ndim != 2:
        raise ValueError(f"curve.gamma returned unexpected shape {pts.shape}")
    # Normalize to (N,3)
    if pts.shape[0] == 3 and pts.shape[1] != 3:
        pts = pts.T
    if pts.shape[1] != 3:
        raise ValueError(f"Expected last dim 3 for xyz; got {pts.shape}")
    return pts

In [ ]:
def _coil_centroid_and_phi(coil, n_samples=200):
    """Sample the coil curve and compute centroid and toroidal angle φ of the centroid."""
    curve = getattr(coil, "curve", coil)  # sometimes `coil` is already a curve
    pts = sample_curve_points(curve, n_samples=n_samples)
    xbar, ybar, zbar = pts[:, 0].mean(), pts[:, 1].mean(), pts[:, 2].mean()
    phi = (np.arctan2(ybar, xbar) + 2*np.pi) % (2*np.pi)
    return xbar, ybar, zbar, phi

def _coil_current_scalar(coil):
    # Use same access you used earlier for consistency. Adjust if needed.
    # Fallbacks (commented) are typical simsopt APIs if your object differs.
    try:
        return float(coil.current.current_to_scale.current)
    except Exception:
        try:
            return float(coil.current.get_value()/(795774.7154594767))
        except Exception:
            return float(coil.current)

In [ ]:
def process_coils(file_path):
    try:
        id = str(file_path)[-12:-5]
        surfaces, coils = load(str(file_path))
        s = surfaces[-1]

        # --- compute per-coil stats ---
        nfp = int(s.nfp)
        sector_width = 2*np.pi / nfp

        stats = []
        for idx, coil in enumerate(coils):
            xbar, ybar, zbar, phi = _coil_centroid_and_phi(coil, n_samples=200)
            k = int(np.floor(phi / sector_width)) % nfp             # sector index
            phi_red = phi - k * sector_width                        # reduced angle in [0, sector_width)
            I = _coil_current_scalar(coil)
            stats.append({
                "idx": idx,
                "phi": phi,
                "phi_red": phi_red,
                "sector": k,
                "zbar": zbar,
                "I": I,
            })

        # --- pick a single sector to remove toroidal copies ---
        chosen_sector = 0
        sel = [st for st in stats if st["sector"] == chosen_sector]
        if not sel:
            # fallback: choose sector with most coils
            counts = np.bincount([st["sector"] for st in stats], minlength=nfp)
            chosen_sector = int(np.argmax(counts))
            sel = [st for st in stats if st["sector"] == chosen_sector]

        # --- canonical sort inside chosen sector ---
        sel.sort(key=lambda st: (st["phi_red"], -st["zbar"], -np.sign(st["I"]), -abs(st["I"])))

        # --- how many coils to keep? ---
        # If your intention was "unique shapes per (nfp*mirror)", selecting one sector already removed nfp.
        # We now KEEP both mirror partners (upper/lower) unless you truly only want one of them.
        num_coils = min(len(sel), MAX_COILS)

        coil_array = np.zeros((MAX_COILS, FEATURES_PER_COIL), dtype=np.float32)
        order_indices = []  # permutation: canonical position -> original coils[] index

        for dst_i in range(num_coils):
            src_idx = sel[dst_i]["idx"]
            # your original packing: first element is current, then last 99 Fourier params
            params = coils[src_idx].x[-99:]  # adjust if your feature layout changes
            curr = np.array(_coil_current_scalar(coils[src_idx]), dtype=np.float32)
            coil_array[dst_i] = np.concatenate([curr[None], params.astype(np.float32)], axis=0)
            order_indices.append(src_idx)

        coil_mask = np.array([1]*num_coils + [0]*(MAX_COILS - num_coils), dtype=np.int64)

        # Optional: return order_indices if you want to log them or store in TFRecord
        return id, coil_array, coil_mask  # , order_indices

    except Exception as e:
        print(f"Failed on {file_path}: {e}")
        return None

In [13]:
'''def process_coils(file_path):
    try:
        id = str(file_path)[-12:-5]
        surfaces, coils = load(str(file_path))
        s = surfaces[-1]

        num_coils = len(coils) // (s.nfp * 2)
        num_coils = min(num_coils, MAX_COILS)

        # log_scaler = np.log10(coils[0].current.scale)
        # scaler_token = np.full((1, FEATURES_PER_COIL), log_scaler, dtype=np.float32)

        coil_array = np.zeros((MAX_COILS, FEATURES_PER_COIL), dtype=np.float32)
        for i in range(num_coils):
            params = coils[i].x[-99:]  # 99 Fourier + 1 current
            curr = np.array(coils[i].current.current_to_scale.current)
            coil_array[i] = np.append(curr, params)

        # coil_array[-1] = scaler_token  # context token

        coil_mask = np.array([1] * num_coils + [0] * (MAX_COILS - num_coils), dtype=np.int64)

        return id, coil_array, coil_mask #serialize_coil(id, coil_array, coil_mask)
        
    except Exception as e:
        print(f"Failed on {file_path}: {e}")
        return None    '''

'def process_coils(file_path):\n    try:\n        id = str(file_path)[-12:-5]\n        surfaces, coils = load(str(file_path))\n        s = surfaces[-1]\n\n        num_coils = len(coils) // (s.nfp * 2)\n        num_coils = min(num_coils, MAX_COILS)\n\n        # log_scaler = np.log10(coils[0].current.scale)\n        # scaler_token = np.full((1, FEATURES_PER_COIL), log_scaler, dtype=np.float32)\n\n        coil_array = np.zeros((MAX_COILS, FEATURES_PER_COIL), dtype=np.float32)\n        for i in range(num_coils):\n            params = coils[i].x[-99:]  # 99 Fourier + 1 current\n            curr = np.array(coils[i].current.current_to_scale.current)\n            coil_array[i] = np.append(curr, params)\n\n        # coil_array[-1] = scaler_token  # context token\n\n        coil_mask = np.array([1] * num_coils + [0] * (MAX_COILS - num_coils), dtype=np.int64)\n\n        return id, coil_array, coil_mask #serialize_coil(id, coil_array, coil_mask)\n        \n    except Exception as e:\n        print

In [12]:
def process_surface(file_path):
    """
    Convert s.x and metadata to [N_modes, 5] array:
    columns = [m_norm, n_norm, is_cos, is_R, coeff_value]
    """
    try:
        id = str(file_path)[-12:-5]
        surfaces, coils = load(str(file_path))
        outer_surface = surfaces[-1]
        s = outer_surface.to_RZFourier() #the bottleneck in speed

        x = s.x  # the coeff vector
        nfp = s.nfp

        num_coefs = len(x)
        if num_coefs > MAX_COEFS:
            print(num_coefs)
        
        m = s.m  # mode numbers, shape (N_modes,)
        n = s.n
        
        num_modes = (len(m)+1)//2
        # Normalize mode indices
        max_m = np.max(np.abs(m)) or 1
        max_n = np.max(np.abs(n)) or 1

        # Type flags
        is_cos = np.concatenate([
            np.ones(num_modes, dtype=bool),  # R_cos
            np.zeros(num_modes-1, dtype=bool)  # Z_sin
        ])

        surface_set = np.zeros((MAX_COEFS+1, 4), dtype=np.float32)
        surface_mask = np.zeros((MAX_COEFS,), dtype=np.int64)
        nfp_norm = float(nfp) / MAX_NFP 

        for i in range(min(num_coefs, MAX_COEFS)):
            surface_set[i] = [
                m[i] / max_m,
                n[i] / max_n,
                float(is_cos[i]),
                x[i]
            ]
            surface_mask[i] = 1

        surface_set[-1] = nfp_norm
            
        return surface_set, surface_mask #serialize_surface(id, surface_set, surface_mask)

    except Exception as e:
        print(f"Failed on {file_path}: {e}")
        return None

In [13]:
def process_file(file_path):
    id, coil_array, coil_mask = process_coils(file_path)

    try: 
        surface_set, surface_mask = process_surface(file_path)
        return serialize_data(id, coil_array, coil_mask, surface_set, surface_mask)
    except Exception as e:
        print(f'{file_path} failure handled.')
        return None

In [14]:
def write_tfrecord_chunk(serialized_examples, output_path):
    with tf.io.TFRecordWriter(str(output_path)) as writer:
        for ex in serialized_examples:
            if ex:
                writer.write(ex)

def datasets_to_tfrecords(directory: Path, output_dir: Path, idx,
                               chunk_size=CHUNK_SIZE, num_workers=NUM_PROCESS_WORKERS):
    files = list(directory.glob("*.json"))
    total_files = len(files)
    output_dir.mkdir(parents=True, exist_ok=True)

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        serialized_examples = list(tqdm(
            executor.map(process_file, files),
            total=total_files,
            desc=f"Chunk {idx//chunk_size:03d}"
        ))

    serialized_examples = [ex for ex in serialized_examples if ex is not None]

    output_path = output_dir / f"surface_coil_chunk_{idx:03d}.tfrecord"

    write_tfrecord_chunk(serialized_examples, output_path)
    print(f"✅ Saved {len(serialized_examples)} surface coil pair samples to {output_dir}")


In [19]:
for idx, chunk in enumerate(chunks, start=0):
    print(f"\n=== Chunk {idx}/{len(chunks)}: {len(chunk)} devices ===")
    results = []

    os.makedirs(SIMSOPT_DIR, exist_ok=True)

    with ThreadPoolExecutor(max_workers=NUM_THREAD_WORKERS) as executor:
        futures = {executor.submit(process_device, dev): dev for dev in chunk}
        for fut in tqdm(as_completed(futures), total=len(futures), desc=f"Chunk {idx}"):
            results.append(fut.result())
    
    datasets_to_tfrecords(directory=data_dir, output_dir=output_dir, idx=idx)

    log_df = pd.concat([log_df, pd.DataFrame(results)], ignore_index=True)
    log_df.to_csv(LOG_CSV, index=False)
    success = sum(r["status"] == "success" for r in results)
    print(f"Chunk {idx} completed: {success}/{len(results)} successful.")

    try:
        shutil.rmtree(SIMSOPT_DIR)
    except OSError as e:
        print(f'Error deleting directory: {e}')


=== Chunk 0/32: 10000 devices ===


Chunk 0: 100%|██████████| 10000/10000 [00:00<00:00, 159159.11it/s]


ScaledCurrent1
-474110.27804410254
795774.7154594767
ScaledCurrent2-173421.57760046108

795774.7154594767
ScaledCurrent3
-1446704.549733777
795774.7154594767
ScaledCurrent4
615659.3276062163
795774.7154594767
ScaledCurrent12
-615659.3276062163
-1.0
Failed on quasr_simsopt_files/input_0029216.json: 'ScaledCurrent' object has no attribute 'current'
ScaledCurrent13
-529964.6638656477
795774.7154594767
ScaledCurrent14
-1061612.5030601579
795774.7154594767
ScaledCurrent18
1061612.5030601579
-1.0
Failed on quasr_simsopt_files/input_0014533.json: 'ScaledCurrent' object has no attribute 'current'
ScaledCurrent19
-785890.4302878877
795774.7154594767
ScaledCurrent20
-777162.4549206169
795774.7154594767
ScaledCurrent24
777162.4549206169
-1.0
Failed on quasr_simsopt_files/input_0011936.json: 'ScaledCurrent' object has no attribute 'current'
ScaledCurrent25
-442116.3756034946
795774.7154594767
ScaledCurrent26
-508184.24917048385
795774.7154594767
ScaledCurrent30
508184.24917048385
-1.0
Failed on qu

Chunk 000:   0%|          | 0/10000 [00:00<?, ?it/s]

ScaledCurrent115
-636707.7612317706

Chunk 000:   0%|          | 0/10000 [00:00<?, ?it/s]


795774.7154594767
ScaledCurrent116
-363659.71078046307
795774.7154594767
ScaledCurrent120
363659.71078046307
-1.0
Failed on quasr_simsopt_files/input_0010587.json: 'ScaledCurrent' object has no attribute 'current'
ScaledCurrent121
-34784.83249089118
795774.7154594767
ScaledCurrent122
-229484.9695716044
795774.7154594767
ScaledCurrent123
-174559.10118729185
795774.7154594767
ScaledCurrent124
-200458.52555982283
795774.7154594767
ScaledCurrent125
-122541.93477075105
795774.7154594767
ScaledCurrent126
-198860.80622157574
795774.7154594767


ScaledCurrent157
-211079.76185047996
795774.7154594767
ScaledCurrent158
-168944.5011542582
795774.7154594767
ScaledCurrent159
-74176.83828056444
795774.7154594767
ScaledCurrent165
74176.83828056444
-1.0
Failed on quasr_simsopt_files/input_0024261.json: 'ScaledCurrent' object has no attribute 'current'
ScaledCurrent166
-837423.7328793893
795774.7154594767
ScaledCurrent169
837423.7328793893
-1.0
Failed on quasr_simsopt_files/input_0042377.json: 'ScaledCurrent' object has no attribute 'current'


TypeError: cannot unpack non-iterable NoneType object